In [1]:
import csv, requests, json, collections, os, pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re
import numpy as np
from numpy.random import randint
regex = re.compile('[^a-zA-Z0-9-_]')
import xlwt 
from xlwt import Workbook
import pandas as pd
from os import listdir
from os.path import isfile, join
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
import stanza
nlp = stanza.Pipeline('en',verbose=False, processors='tokenize,pos,lemma')

In [2]:
category_properties = {}
category_properties['Hardware-Devices'] = ['notRelevant','isPowered','isConnected','isSetup','isUsed']
category_properties['Software-Device-Drivers'] = ['notRelevant','isInstalled','isRelatedDeviceConnected','isSettingsChanged']
category_properties['Software-OS-Related'] = ['notRelevant','isOpened','isSettingsChanged']
category_properties['Software-Other'] = ['notRelevant','isInstalled','isOpened','isSettingsChanged']
category_properties['Hardware-Other'] = ['']

property_event = {}
property_event['notRelevant'] = True
property_event['isPowered'] = False
property_event['isConnected'] = False
property_event['isSetup'] = True
property_event['isUsed'] = True
property_event['isInstalled'] = False
property_event['isRelatedDeviceConnected'] = False
property_event['isSettingsChanged'] = True
property_event['isOpened'] = False

property_values = {}
property_values[True] = ['true','false']
property_values[False] = ['f->t','t->f','noChange','start-T','start-F']

In [3]:
step_details = []
mentions = {}
entities = {}
ent_variants = {}
ent_verb_spans = {}
merged_ents = {}
step_wise_properties = {}
ent_wise_spans = {}

all_annotations = pickle.load(open('./annotations/all_files_v1.pkl','rb'))

In [4]:
# def process_word(word):
#     w = word
#     cap = False
#     if(w.isupper()):
#         cap = True
#     w = w.lower()
# #     print(w)
#     if('\'s' == w):
#         w=''
#     if('cd' == w):
#         w='CD'
#     w = lemmatizer.lemmatize(w)
# #     print(w)
#     if(not cap):
#         w = w.capitalize()
#     else:
#         w = w.upper()
#     w = regex.sub('', w)
#     if(len(w)<2 and w!= 'X'):
#         w=''
#     return w

def get_line(offset):
    for i, step in enumerate(step_details):
        if(int(offset)<step[2]):
            return i-2
    return len(step_details)-1
def clean_text(ent_text):
    tokens=nlp(ent_text)
    tokens = [w for sent in tokens.sentences for w in sent.words]
    lemmas = [token.lemma.lower().strip() for token in tokens]
    ent_text = ' '.join(lemmas)
    return ent_text
def get_text(mention):
    text = mentions[mention][-2]
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text
def get_base_ent(text):
    while(entities[text][0] != '' and entities[text][0] != text):
        text = entities[text][0]
    return text
def state_change(new_state, prev_state):
    if(new_state==prev_state):
        return 'None'
    elif(new_state=='True'):
        return 'True'
    elif(new_state=='False'):
        return 'False'
    else:
        return 'None'
def get_indexes(sent_tokens, ent_text,verb=False):
    indexes = ['0' for _ in sent_tokens]
    if verb:
        indexes = ['1' if token.pos=='VERB' else '0' for token in sent_tokens]
    else:
        for ent in ent_variants[ent_text]:
            ent_tokens = nlp(ent)
            ent_tokens = [w for sent in ent_tokens.sentences for w in sent.words]
            for i in range(len(sent_tokens)-len(ent_tokens)+1):
                if(all(ent_tokens[j].lemma.lower().strip()==sent_tokens[i+j].lemma.lower().strip() for j in range(len(ent_tokens)))):
                    for j in range(len(ent_tokens)):
                        indexes[i+j]='1'
    return indexes
def process_sent(sent:str,ent_text:str):
    tokens=nlp(sent)
    tokens = [w for sent in tokens.sentences for w in sent.words]
    lemmas = [token.lemma.lower().strip() for token in tokens]
    ent_spans = get_indexes(tokens,ent_text)
    verb_spans = get_indexes(tokens, ent_text, verb=True)
    return [lemmas,ent_spans,verb_spans]

In [5]:
def parse_annotations(out_file,doc_id=1,folder='config/',file='monitor',filter_prop=''):
    global step_details, mentions,entities,ent_variants,ent_verb_spans,merged_ents,step_wise_properties,ent_wise_spans
    try:
        steps = all_annotations[file]['steps']
        step_details = all_annotations[file]['step_details']
        mentions = all_annotations[file]['mentions']
        entities = all_annotations[file]['entities']
        ent_variants = all_annotations[file]['ent_variants']
        ent_verb_spans = all_annotations[file]['ent_verb_spans']
        merged_ents = all_annotations[file]['merged_ents']
        step_wise_properties = all_annotations[file]['step_wise_properties']
        for ent in step_wise_properties:
            for prop in step_wise_properties[ent]:
                if(filter_prop != '' and prop!=filter_prop):
                    continue
                for step_no in range(len(steps)-1):
                    sent_lemmas, ent_spans, verb_spans = ent_verb_spans[ent][step_no]
#                     if(all(ent_tag=='0' for ent_tag in ent_spans)):
#                         continue;
    #                 print(sent_lemmas, ent_spans, verb_spans)
                    if(property_event[prop]):
                        print('####'.join(sent_lemmas),','.join(verb_spans),','.join(ent_spans),step_wise_properties[ent][prop][step_no+1],sep='\t',file=out_file)
                    else:
                        print('####'.join(sent_lemmas),','.join(verb_spans),','.join(ent_spans),state_change(step_wise_properties[ent][prop][step_no+1],step_wise_properties[ent][prop][step_no]),sep='\t',file=out_file)
    except Exception as e:
        print(e)

In [6]:
# parse_annotations()
# f.close()

In [7]:
def parse_prop(filter_prop='isOpened'):
    np.random.seed(0)
    print('\n\n',filter_prop)
    train_f = open('./annotations/prolocal_annotations_'+filter_prop+'_train.tsv','w',encoding='utf-8')
    dev_f = open('./annotations/prolocal_annotations_'+filter_prop+'_dev.tsv','w',encoding='utf-8')
#     test_f = open('./annotations/prolocal_annotations_'+filter_prop+'_test.tsv','w',encoding='utf-8')
    data_folder = './annotations/'
#     files = [train_f,dev_f,test_f]
    count = 1
    for folder in [fol for fol in listdir(data_folder) if not isfile(join(data_folder, fol))]:
        test_f = open('./annotations/prolocal_annotations_'+folder+'_'+filter_prop+'_test.tsv','w',encoding='utf-8')
        files = [train_f,dev_f,test_f]
        folder = data_folder + folder+'/'
        onlyfiles = [f[:-4] for f in listdir(folder) if f.endswith('.txt')]
        for file in onlyfiles:
            out_file = np.random.choice(files,p=[0.8,0.1,0.1])
            print(count, folder, file,out_file.name[-10:])
            parse_annotations(out_file=out_file,doc_id=count, folder=folder, file=file, filter_prop=filter_prop)
            count+=1
        test_f.close()
    train_f.close()
    dev_f.close()
    test_f.close()

In [12]:
parse_prop()



 isOpened
1 ./annotations/Controllers/ 199_1___Test_Your_PS4_Controller___Steps _train.tsv
2 ./annotations/Controllers/ 200_1___Connect_an_Xbox_One_Controller_to_an_Xbox_One___Connecting_Wirelessly _train.tsv
3 ./annotations/Controllers/ 201_1___Disable_the_Green_Light_on_a_Thrustmaster_T.16000M_Joystick___Steps _train.tsv
4 ./annotations/Controllers/ 202_1___Hack_an_Xbox_Controller_Into_a_PC_Gamepad___Steps _train.tsv
5 ./annotations/Controllers/ 203_1___Connect_a_Razer_Controller_to_a_PC___Connecting_Wirelessly _train.tsv
6 ./annotations/Controllers/ 204_1___Use_the_Razer_Hydra___Installing_the_Hydra _train.tsv
7 ./annotations/Controllers/ 204_2___Use_the_Razer_Hydra___Using_the_Hydra _train.tsv
8 ./annotations/Headphones/ 161_1___Repair_Dodgy_or_Broken_Headphones___Finding_the_Problem ed_dev.tsv
9 ./annotations/Headphones/ 161_2___Repair_Dodgy_or_Broken_Headphones___Fixing_the_Cable d_test.tsv
10 ./annotations/Headphones/ 161_3___Repair_Dodgy_or_Broken_Headphones___Fixing_a_Broken

188 ./annotations/Monitors/ 16_1___Clean_a_Flat_Panel_Monitor___Steps _train.tsv
189 ./annotations/Monitors/ 17_1___Extend_a_Netbook_to_an_External_Monitor___Steps _train.tsv
190 ./annotations/Monitors/ 1_1___Connect_a_Laptop_to_a_Monitor___Connecting_the_Monitor _train.tsv
191 ./annotations/Monitors/ 1_2___Connect_a_Laptop_to_a_Monitor___Detecting_a_Display_in_Windows _train.tsv
192 ./annotations/Monitors/ 1_3___Connect_a_Laptop_to_a_Monitor___Detecting_a_Display_in_MacOS _train.tsv
193 ./annotations/Monitors/ 1_4___Connect_a_Laptop_to_a_Monitor___Adjusting_Display_Settings_on_Windows _train.tsv
194 ./annotations/Monitors/ 1_5___Connect_a_Laptop_to_a_Monitor___Adjusting_Display_Settings_on_Mac d_test.tsv
195 ./annotations/Monitors/ 2_1___Switch_Monitors_1_and_2_on_a_PC__Steps _train.tsv
196 ./annotations/Monitors/ 3_1___Use_Your_TV_as_a_Second_Monitor_for_Your_Computer___Connecting_Your_Computer_to_the_TV _train.tsv
197 ./annotations/Monitors/ 3_2___Use_Your_TV_as_a_Second_Monitor_for

368 ./annotations/Ubuntu/ 649_4___Install_Windows_from_Ubuntu___Setting_Up_Dual_Boot _train.tsv
369 ./annotations/Ubuntu/ 650_1___Format_a_USB_Flash_Drive_in_Ubuntu___Using_the_Disks_Utility d_test.tsv
370 ./annotations/Ubuntu/ 650_2___Format_a_USB_Flash_Drive_in_Ubuntu___Using_the_Terminal _train.tsv
371 ./annotations/Ubuntu/ 651_1___Become_Root_in_Ubuntu___Running_Root_Commands_with_Sudo _train.tsv
372 ./annotations/Ubuntu/ 651_2___Become_Root_in_Ubuntu___Enabling_the_Root_User _train.tsv
373 ./annotations/Ubuntu/ 652_1___Install_Ubuntu_Linux___Preparing_to_Install _train.tsv
374 ./annotations/Ubuntu/ 652_2___Install_Ubuntu_Linux___Installing_on_Windows _train.tsv
375 ./annotations/Ubuntu/ 652_3___Install_Ubuntu_Linux___Installing_on_Mac _train.tsv
376 ./annotations/Ubuntu/ 653_1___Change_Keyboard_Layout_in_Ubuntu___Steps _train.tsv
377 ./annotations/Ubuntu/ 654_1___Set_up_an_FTP_Server_in_Ubuntu_Linux___Installing_the_FTP_Framework _train.tsv
378 ./annotations/Ubuntu/ 654_2___Set_up

In [9]:
# parse_prop('isPowered')
# parse_prop('isConnected')
# parse_prop('isSetup')
# parse_prop('isUsed')
# parse_prop('isInstalled')
# parse_prop('isRelatedDeviceConnected')
# parse_prop('isSettingsChanged')
# parse_prop('isOpened')



 isPowered
1 ./annotations/Controllers/ 199_1___Test_Your_PS4_Controller___Steps _train.tsv
2 ./annotations/Controllers/ 200_1___Connect_an_Xbox_One_Controller_to_an_Xbox_One___Connecting_Wirelessly _train.tsv
3 ./annotations/Controllers/ 201_1___Disable_the_Green_Light_on_a_Thrustmaster_T.16000M_Joystick___Steps _train.tsv
4 ./annotations/Controllers/ 202_1___Hack_an_Xbox_Controller_Into_a_PC_Gamepad___Steps _train.tsv
5 ./annotations/Controllers/ 203_1___Connect_a_Razer_Controller_to_a_PC___Connecting_Wirelessly _train.tsv
6 ./annotations/Controllers/ 204_1___Use_the_Razer_Hydra___Installing_the_Hydra _train.tsv
7 ./annotations/Controllers/ 204_2___Use_the_Razer_Hydra___Using_the_Hydra _train.tsv
8 ./annotations/Headphones/ 161_1___Repair_Dodgy_or_Broken_Headphones___Finding_the_Problem ed_dev.tsv
9 ./annotations/Headphones/ 161_2___Repair_Dodgy_or_Broken_Headphones___Fixing_the_Cable d_test.tsv
10 ./annotations/Headphones/ 161_3___Repair_Dodgy_or_Broken_Headphones___Fixing_a_Broke

255 ./annotations/OSX/ 1146_1___Install_macOS_on_a_Windows_PC___Preparing_to_Install d_test.tsv
256 ./annotations/OSX/ 1146_2___Install_macOS_on_a_Windows_PC___Downloading_Unibeast _train.tsv
257 ./annotations/OSX/ 1146_3___Install_macOS_on_a_Windows_PC___Downloading_the_macOS_High_Sierra_Installer _train.tsv
258 ./annotations/OSX/ 1146_4___Install_macOS_on_a_Windows_PC___Formatting_Your_USB_Drive _train.tsv
259 ./annotations/OSX/ 1146_5___Install_macOS_on_a_Windows_PC___Creating_the_Unibeast_Installation_Tool d_test.tsv
260 ./annotations/OSX/ 1146_6___Install_macOS_on_a_Windows_PC___Changing_Windows_Boot_Order _train.tsv
261 ./annotations/OSX/ 1146_7___Install_macOS_on_a_Windows_PC___Installing_macOS_on_Your_PC _train.tsv
262 ./annotations/OSX/ 1146_8___Install_macOS_on_a_Windows_PC___Enabling_Drivers_with_Multibeast _train.tsv
263 ./annotations/OSX/ 1147_1___Download_YouTube_Videos_on_a_Mac___Using_QuickTime _train.tsv
264 ./annotations/OSX/ 1147_2___Download_YouTube_Videos_on_a_Mac_

490 ./annotations/Windows10/ 737_1___LoginAsanAdministratorinWindows10Steps _train.tsv
491 ./annotations/Windows10/ 738_1___Fix_Microsoft_Store_Downloading_Problems___ _train.tsv
492 ./annotations/Windows10/ 738_2___Fix_Microsoft_Store_Downloading_Problems___Updating_Your_Current_Apps _train.tsv
493 ./annotations/Windows10/ 738_3___Fix_Microsoft_Store_Downloading_Problems___Signing_Out_of_the_Microsoft_Store ed_dev.tsv
494 ./annotations/Windows10/ 738_4___Fix_Microsoft_Store_Downloading_Problems___ _train.tsv
495 ./annotations/Windows10/ 739_1___Eject_the_CD_Tray_for_Windows_10___Using_Windows _train.tsv
496 ./annotations/Windows10/ 739_2___Eject_the_CD_Tray_for_Windows_10___Using_a_Paperclip_if_the_Drive_is_Stuck _train.tsv
497 ./annotations/Windows10/ 739_3___Eject_the_CD_Tray_for_Windows_10___Ejecting_from_Inside_a_Desktop_PC _train.tsv
498 ./annotations/Windows10/ 740_1___Screenshot_in_Windows_10___ _train.tsv
499 ./annotations/Windows10/ 740_2___Screenshot_in_Windows_10___Using_Pr

194 ./annotations/Monitors/ 1_5___Connect_a_Laptop_to_a_Monitor___Adjusting_Display_Settings_on_Mac d_test.tsv
195 ./annotations/Monitors/ 2_1___Switch_Monitors_1_and_2_on_a_PC__Steps _train.tsv
196 ./annotations/Monitors/ 3_1___Use_Your_TV_as_a_Second_Monitor_for_Your_Computer___Connecting_Your_Computer_to_the_TV _train.tsv
197 ./annotations/Monitors/ 3_2___Use_Your_TV_as_a_Second_Monitor_for_Your_Computer___Changing_Display_Settings_on_Windows _train.tsv
198 ./annotations/Monitors/ 3_3___Use_Your_TV_as_a_Second_Monitor_for_Your_Computer___Changing_Display_Settings_on_Mac _train.tsv
199 ./annotations/Monitors/ 4_1___Fix_a_Stuck_Pixel_on_an_LCD_Monitor___Preparing_to_Fix _train.tsv
200 ./annotations/Monitors/ 4_2___Fix_a_Stuck_Pixel_on_an_LCD_Monitor___Using_Screen-Fixing_Software _train.tsv
201 ./annotations/Monitors/ 4_3___Fix_a_Stuck_Pixel_on_an_LCD_Monitor___Using_Pressure_and_Heat _train.tsv
202 ./annotations/Monitors/ 5_1___Repair_LCD_Monitors___Diagnosing_the_Problem _train.tsv


425 ./annotations/Windows/ 265_1___Take_a_Screenshot_in_Microsoft_Windows___Taking_a_FullScreen_Screenshot_on_Any_Windows_Computer _train.tsv
426 ./annotations/Windows/ 265_2___Take_a_Screenshot_in_Microsoft_Windows___Taking_a_OneWindow_Screenshot _train.tsv
427 ./annotations/Windows/ 265_3___Take_a_Screenshot_in_Microsoft_Windows___Using_the_Snipping_Tool_Program d_test.tsv
428 ./annotations/Windows/ 265_4___Take_a_Screenshot_in_Microsoft_Windows___Using_the_Snipping_Tool_Shortcut _train.tsv
429 ./annotations/Windows/ 265_5___Take_a_Screenshot_in_Microsoft_Windows___Screenshotting_Several_Windows_in_a_Row _train.tsv
430 ./annotations/Windows/ 266_1___Zoom_in_on_a_PC___Zooming_on_the_Whole_Screen_with_Magnifier _train.tsv
431 ./annotations/Windows/ 266_2___Zoom_in_on_a_PC___Using_Web_Browser_Zoom_Features _train.tsv
432 ./annotations/Windows/ 267_1___Create_a_Windows_Icon___Creating_an_Icon_in_ICO_Convert _train.tsv
433 ./annotations/Windows/ 267_2___Create_a_Windows_Icon___Creating_an

...

122 ./annotations/Linux/ 555_4___Create_and_Edit_Text_File_in_Linux_by_Using_Terminal___Using_Emacs _train.tsv
123 ./annotations/Linux/ 556_1___Check_the_IP_Address_in_Linux___Finding_Your_Public_IP_Address d_test.tsv
124 ./annotations/Linux/ 556_2___Check_the_IP_Address_in_Linux___Finding_Your_Private_IP_Address _train.tsv
125 ./annotations/Linux/ 557_1___Take_a_Screenshot_in_Linux___Using_Gnome_Screenshot _train.tsv
126 ./annotations/Linux/ 557_2___Take_a_Screenshot_in_Linux___Using_GIMP _train.tsv
127 ./annotations/Linux/ 557_3___Take_a_Screenshot_in_Linux___Using_ImageMagick _train.tsv
128 ./annotations/Linux/ 557_4___Take_a_Screenshot_in_Linux___Using_Shutter _train.tsv
129 ./annotations/Linux/ 558_1___Add_or_Change_the_Default_Gateway_in_Linux___Editing_Your_Configuration_File _train.tsv
130 ./annotations/Linux/ 559_1___Find_a_File_in_Linux___ _train.tsv
131 ./annotations/Linux/ 560_1___Install_XAMPP_on_Linux___Installing_XAMPP _train.tsv
132 ./annotations/Linux/ 560_2___Install_

329 ./annotations/Printers/ 37_2___3D_Print_with_CURA_on_Creality%27s_Ender_3___Loading_the_Filament _train.tsv
330 ./annotations/Printers/ 37_3___3D_Print_with_CURA_on_Creality%27s_Ender_3___Printing_the_Object _train.tsv
331 ./annotations/Printers/ 38_1___Put_Ink_Cartridges_in_a_Printer___Putting_Ink_Cartridges_in_a_HP_Printer _train.tsv
332 ./annotations/Printers/ 38_3___Put_Ink_Cartridges_in_a_Printer___Putting_Ink_Cartridges_in_an_Epson_Printer _train.tsv
333 ./annotations/Printers/ 39_1___Reset_an_HP_Photosmart_Printer___Resetting_the_Printer _train.tsv
334 ./annotations/Printers/ 40_1___Replace_a_Toner_Cartridge_in_a_Laser_Printer___Steps _train.tsv
335 ./annotations/Scanners/ 185_1___Scan_Documents_Into_PDF___On_Windows _train.tsv
336 ./annotations/Scanners/ 185_2___Scan_Documents_Into_PDF___On_Mac _train.tsv
337 ./annotations/Scanners/ 185_3___Scan_Documents_Into_PDF___Converting_a_Scan_Image_File_into_a_PDF _train.tsv
338 ./annotations/Scanners/ 186_1___Email_a_Scanned_Docume

15 ./annotations/Headphones/ 164_2___Keep_Earbuds_from_Falling_Out_of_Your_Ears___Purchasing_Well-Fitting_Earbuds _train.tsv
16 ./annotations/Headphones/ 165_1___Clean_a_Headphone_Jack___Cleaning_with_Cotton_Swabs _train.tsv
17 ./annotations/Headphones/ 166_1___Wear_Headphones___Wearing_In-Ear_Headphones _train.tsv
18 ./annotations/Headphones/ 167_1___Connect_a_HyperX_Cloud_Headset___Connecting_to_a_PC ed_dev.tsv
19 ./annotations/Headphones/ 168_1___Pair_Jaybird_Headphones_on_PC_or_Mac___Pairing_Jaybird_Headphones_with_Windows_Bluetooth _train.tsv
20 ./annotations/Headphones/ 169_1___Use_Computer_Headphones___Plugging_in_Headphones ed_dev.tsv
21 ./annotations/Headphones/ 170_1___Burn_in_Headphones___Preparing_a_Burn-in_Playlist d_test.tsv
22 ./annotations/Headphones/ 171_1___Keep_Your_Headphones_From_Tangling___Steps _train.tsv
23 ./annotations/Headphones/ 172_1___Store_Headphones___Wrapping_the_Cord_Properly _train.tsv
24 ./annotations/Headphones/ 173_1___Test_Earphones___Judging_Soun

215 ./annotations/OS/ 223_1___Open_EXE_Files___ _train.tsv
216 ./annotations/OS/ 223_2___Open_EXE_Files___ _train.tsv
217 ./annotations/OS/ 224_1___Format_a_PC___Formatting_Your_Primary_Drive _train.tsv
218 ./annotations/OS/ 224_2___Format_a_PC___Formatting_a_Secondary_Drive _train.tsv
219 ./annotations/OS/ 224_3___Format_a_PC___Shrinking_an_Existing_Drive _train.tsv
220 ./annotations/OS/ 224_4___Format_a_PC___Securely_Formatting_a_Drive ed_dev.tsv
221 ./annotations/OS/ 225_1___Install_an_Operating_System_on_a_Brand_New_Computer___On_Windows _train.tsv
222 ./annotations/OS/ 225_2___Install_an_Operating_System_on_a_Brand_New_Computer___On_Mac ed_dev.tsv
223 ./annotations/OS/ 226_1___Install_a_New_Operating_System_on_Your_Computer___Determining_Which_Operating_System_to_Install _train.tsv
224 ./annotations/OS/ 226_2___Install_a_New_Operating_System_on_Your_Computer___Installing_Your_New_Operating_System _train.tsv
225 ./annotations/OS/ 226_3___Install_a_New_Operating_System_on_Your_Compu

406 ./annotations/Webcams/ 209_1___Record_a_Video_on_PC__Steps _train.tsv
407 ./annotations/Webcams/ 210_1___Buy_a_Webcam___Steps _train.tsv
408 ./annotations/Webcams/ 211_1___Improve_Your_Webcam%27s_Output_Quality___Steps _train.tsv
409 ./annotations/Webcams/ 212_1___Stream_Your_Webcam___Encoding_a_Stream_with_OBS_Studio _train.tsv
410 ./annotations/Webcams/ 212_2___Stream_Your_Webcam___Streaming_Directly_from_Your_Webcam_on_YouTube _train.tsv
411 ./annotations/Webcams/ 212_3___Stream_Your_Webcam___Streaming_Directly_from_Your_Webcam_on_Facebook _train.tsv
412 ./annotations/Webcams/ 213_1___Webcam_With_Another_Person___Using_a_Webcam_with_Skype _train.tsv
413 ./annotations/Webcams/ 214_1___Add_a_Webcam_to_OBS_on_PC_or_Mac___Steps _train.tsv
414 ./annotations/Webcams/ 215_1___Choose_a_Site_to_Webcam_Model_For___Steps _train.tsv
415 ./annotations/Webcams/ 216_1___Make_a_Webcam_Into_an_Infrared_Camera___Steps _train.tsv
416 ./annotations/Webcams/ 217_1___Use_Webcam_on_Windows_Movie_Maker

In [10]:
# parse_annotations(open('temp.txt','w',encoding='utf-8'),169,'./annotations/Mac/','1017_1___Customize_Your_Mac_Using_the_Terminal___Using_Terminal_Commands','isOpened')

In [11]:
# all_annotations['199_1___Test_Your_PS4_Controller___Steps'].keys()